In [ ]:
import os
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
cd /content/drive/MyDrive/weather_bigdata_contest/code

/content/drive/MyDrive/weather_bigdata_contest/code


In [ ]:
import pandas as pd
import datetime as dt

In [ ]:
sale_df = pd.read_csv('../data/sale_data.csv')
weather_df = pd.read_csv('../data/weather_data.csv', encoding = 'CP949')

In [ ]:
sale_df['대분류_소분류'] = sale_df['대분류'] + '_' + sale_df['소분류']
sale_df.head()

,날짜,대분류,소분류,20대 남성 판매량(개),20대 여성 판매량(개),30대 남성 판매량(개),30대 여성 판매량(개),40대 남성 판매량(개),40대 여성 판매량(개),50대 남성 판매량(개),50대 여성 판매량(개),60대 남성 판매량(개),60대 여성 판매량(개),합계,대분류_소분류
0,2018-01-01,식품,가공란,13.0,37.0,6.0,16.0,6.0,9.0,0.0,3.0,0.0,0.0,90.0,식품_가공란
1,2018-01-01,냉난방가전,가열식 가습기,1.0,1.0,0.0,0.0,2.0,3.0,0.0,0.0,0.0,0.0,7.0,냉난방가전_가열식 가습기
2,2018-01-01,식품,가자미,0.0,3.0,2.0,5.0,1.0,4.0,3.0,2.0,1.0,0.0,21.0,식품_가자미
3,2018-01-01,식품,갈비/찜/바비큐용 돈육,0.0,7.0,7.0,4.0,1.0,2.0,0.0,1.0,0.0,0.0,22.0,식품_갈비/찜/바비큐용 돈육
4,2018-01-01,식품,갈비용 우육,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,2.0,식품_갈비용 우육


In [ ]:
#병합하려고 했는데 날짜 column name이 달라서 '날짜'로 통일
weather_df.rename(columns = {'일시':'날짜'}, inplace = True)

In [ ]:
#pandas의 merge_asof method 사용하여 '날짜'를 기준으로 합치는 방법 사용하려고 했는데, '날짜'의 type이 str이라서 integer나 float으로 변경 필요
#pandas의 to_datetime method를 사용하여 type 변경
sale_df['날짜'] = pd.to_datetime(sale_df['날짜'], format='%Y-%m-%d')
sale_df = sale_df.sort_values('날짜')
weather_df['날짜'] = pd.to_datetime(weather_df['날짜'], format = '%Y-%m-%d')
weather_df = weather_df.sort_values('날짜')

In [ ]:
#df 순서를 바꾸면 안됨
sale_and_weather_df = pd.merge_asof(sale_df, weather_df, on='날짜', allow_exact_matches=True)
sale_and_weather_df.head()

,날짜,대분류,소분류,20대 남성 판매량(개),20대 여성 판매량(개),30대 남성 판매량(개),30대 여성 판매량(개),40대 남성 판매량(개),40대 여성 판매량(개),50대 남성 판매량(개),50대 여성 판매량(개),60대 남성 판매량(개),60대 여성 판매량(개),합계,대분류_소분류,평균기온(°C),최저기온(°C),최고기온(°C),1시간 최다강수량(mm),일강수량(mm),평균 풍속(m/s),풍정합(100m),최소 상대습도(%),평균 상대습도(%),O3,PM10,PM25,최다풍향북북동,최다풍향북동,최다풍향동북동,최다풍향동,최다풍향동남동,최다풍향남동,최다풍향남남동,최다풍향남,최다풍향남남서,최다풍향남서,최다풍향서남서,최다풍향서,최다풍향서북서,...,1시간 최다 강수량 시각(hh)9.0,1시간 최다 강수량 시각(hh)10.0,1시간 최다 강수량 시각(hh)11.0,1시간 최다 강수량 시각(hh)12.0,1시간 최다 강수량 시각(hh)13.0,1시간 최다 강수량 시각(hh)14.0,1시간 최다 강수량 시각(hh)15.0,1시간 최다 강수량 시각(hh)16.0,1시간 최다 강수량 시각(hh)17.0,1시간 최다 강수량 시각(hh)18.0,1시간 최다 강수량 시각(hh)19.0,1시간 최다 강수량 시각(hh)20.0,1시간 최다 강수량 시각(hh)21.0,1시간 최다 강수량 시각(hh)22.0,1시간 최다 강수량 시각(hh)23.0,최소 상대습도 시각(hh)0,최소 상대습도 시각(hh)1,최소 상대습도 시각(hh)2,최소 상대습도 시각(hh)3,최소 상대습도 시각(hh)4,최소 상대습도 시각(hh)5,최소 상대습도 시각(hh)6,최소 상대습도 시각(hh)7,최소 상대습도 시각(hh)8,최소 상대습도 시각(hh)9,최소 상대습도 시각(hh)10,최소 상대습도 시각(hh)11,최소 상대습도 시각(hh)12,최소 상대습도 시각(hh)13,최소 상대습도 시각(hh)14,최소 상대습도 시각(hh)15,최소 상대습도 시각(hh)16,최소 상대습도 시각(hh)17,최소 상대습도 시각(hh)18,최소 상대습도 시각(hh)19,최소 상대습도 시각(hh)20,최소 상대습도 시각(hh)21,최소 상대습도 시각(hh)22,최소 상대습도 시각(hh)23,강수 계속시간(hr)
0,2018-01-01,식품,가공란,13.0,37.0,6.0,16.0,6.0,9.0,0.0,3.0,0.0,0.0,90.0,식품_가공란,0.066157,-3.463993,4.734708,0.0,0.0,1.844893,1590.217336,20.763179,41.267581,0.015544,44.032069,20.229672,0.061075,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.050179,0.559225,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.766846,0.123827,0.048253,0.061075,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2018-01-01,식품,옥수수,0.0,4.0,3.0,6.0,1.0,7.0,1.0,1.0,0.0,0.0,23.0,식품_옥수수,0.066157,-3.463993,4.734708,0.0,0.0,1.844893,1590.217336,20.763179,41.267581,0.015544,44.032069,20.229672,0.061075,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.050179,0.559225,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.766846,0.123827,0.048253,0.061075,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2018-01-01,식품,옥돔,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,2.0,식품_옥돔,0.066157,-3.463993,4.734708,0.0,0.0,1.844893,1590.217336,20.763179,41.267581,0.015544,44.032069,20.229672,0.061075,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.050179,0.559225,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.766846,0.123827,0.048253,0.061075,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2018-01-01,식품,오징어,1.0,2.0,13.0,16.0,7.0,15.0,1.0,3.0,0.0,0.0,58.0,식품_오징어,0.066157,-3.463993,4.734708,0.0,0.0,1.844893,1590.217336,20.763179,41.267581,0.015544,44.032069,20.229672,0.061075,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.050179,0.559225,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.766846,0.123827,0.048253,0.061075,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2018-01-01,식품,오이/가지,1.0,1.0,5.0,8.0,5.0,6.0,2.0,1.0,0.0,0.0,29.0,식품_오이/가지,0.066157,-3.463993,4.734708,0.0,0.0,1.844893,1590.217336,20.763179,41.267581,0.015544,44.032069,20.229672,0.061075,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.050179,0.559225,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.766846,0.123827,0.048253,0.061075,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
#corr 구할 때 필요없는 컬럼 제거
sale_and_weather_df.drop(list(sale_and_weather_df.columns[:13]), axis = 1, inplace = True)
sale_and_weather_df.head()

,합계,대분류_소분류,평균기온(°C),최저기온(°C),최고기온(°C),1시간 최다강수량(mm),일강수량(mm),평균 풍속(m/s),풍정합(100m),최소 상대습도(%),평균 상대습도(%),O3,PM10,PM25,최다풍향북북동,최다풍향북동,최다풍향동북동,최다풍향동,최다풍향동남동,최다풍향남동,최다풍향남남동,최다풍향남,최다풍향남남서,최다풍향남서,최다풍향서남서,최다풍향서,최다풍향서북서,최다풍향북서,최다풍향북북서,최다풍향북,최고기온 시각(hh)0,최고기온 시각(hh)1,최고기온 시각(hh)2,최고기온 시각(hh)3,최고기온 시각(hh)4,최고기온 시각(hh)5,최고기온 시각(hh)6,최고기온 시각(hh)7,최고기온 시각(hh)8,최고기온 시각(hh)9,...,1시간 최다 강수량 시각(hh)9.0,1시간 최다 강수량 시각(hh)10.0,1시간 최다 강수량 시각(hh)11.0,1시간 최다 강수량 시각(hh)12.0,1시간 최다 강수량 시각(hh)13.0,1시간 최다 강수량 시각(hh)14.0,1시간 최다 강수량 시각(hh)15.0,1시간 최다 강수량 시각(hh)16.0,1시간 최다 강수량 시각(hh)17.0,1시간 최다 강수량 시각(hh)18.0,1시간 최다 강수량 시각(hh)19.0,1시간 최다 강수량 시각(hh)20.0,1시간 최다 강수량 시각(hh)21.0,1시간 최다 강수량 시각(hh)22.0,1시간 최다 강수량 시각(hh)23.0,최소 상대습도 시각(hh)0,최소 상대습도 시각(hh)1,최소 상대습도 시각(hh)2,최소 상대습도 시각(hh)3,최소 상대습도 시각(hh)4,최소 상대습도 시각(hh)5,최소 상대습도 시각(hh)6,최소 상대습도 시각(hh)7,최소 상대습도 시각(hh)8,최소 상대습도 시각(hh)9,최소 상대습도 시각(hh)10,최소 상대습도 시각(hh)11,최소 상대습도 시각(hh)12,최소 상대습도 시각(hh)13,최소 상대습도 시각(hh)14,최소 상대습도 시각(hh)15,최소 상대습도 시각(hh)16,최소 상대습도 시각(hh)17,최소 상대습도 시각(hh)18,최소 상대습도 시각(hh)19,최소 상대습도 시각(hh)20,최소 상대습도 시각(hh)21,최소 상대습도 시각(hh)22,최소 상대습도 시각(hh)23,강수 계속시간(hr)
0,90.0,식품_가공란,0.066157,-3.463993,4.734708,0.0,0.0,1.844893,1590.217336,20.763179,41.267581,0.015544,44.032069,20.229672,0.061075,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.050179,0.559225,0.062101,0.123827,0.143592,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.766846,0.123827,0.048253,0.061075,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,23.0,식품_옥수수,0.066157,-3.463993,4.734708,0.0,0.0,1.844893,1590.217336,20.763179,41.267581,0.015544,44.032069,20.229672,0.061075,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.050179,0.559225,0.062101,0.123827,0.143592,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.766846,0.123827,0.048253,0.061075,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2.0,식품_옥돔,0.066157,-3.463993,4.734708,0.0,0.0,1.844893,1590.217336,20.763179,41.267581,0.015544,44.032069,20.229672,0.061075,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.050179,0.559225,0.062101,0.123827,0.143592,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.766846,0.123827,0.048253,0.061075,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,58.0,식품_오징어,0.066157,-3.463993,4.734708,0.0,0.0,1.844893,1590.217336,20.763179,41.267581,0.015544,44.032069,20.229672,0.061075,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.050179,0.559225,0.062101,0.123827,0.143592,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.766846,0.123827,0.048253,0.061075,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,29.0,식품_오이/가지,0.066157,-3.463993,4.734708,0.0,0.0,1.844893,1590.217336,20.763179,41.267581,0.015544,44.032069,20.229672,0.061075,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.050179,0.559225,0.062101,0.123827,0.143592,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.766846,0.123827,0.048253,0.061075,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
#품목 카테고리
category_array = sale_and_weather_df['대분류_소분류'].unique()

In [ ]:
#품목 카테고리를 넣으면 총 판매량과 날씨 상관관계 구해주는 함수 정의
def get_corr(category):
    temp_df = sale_and_weather_df[sale_and_weather_df['대분류_소분류'] == category]
    temp_df.drop('대분류_소분류', axis = 1, inplace = True)
    return temp_df.corr().iloc[0, 1:].values

In [ ]:
#예시
get_corr('식품_가공란')

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4174: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


array([-0.01272747, -0.02370395, -0.0034906 ,  0.03322689, -0.00196646,
       -0.03286275, -0.03392248, -0.07220295, -0.04538795,  0.08623826,
        0.14961498,  0.1582681 , -0.00230737, -0.04198548, -0.01281418,
       -0.13586491, -0.07462175,  0.00395092,  0.03175213,  0.0010969 ,
        0.05145786,  0.05092013,  0.0831845 ,  0.07285717,  0.01818696,
       -0.01156842,  0.03034841, -0.07799659, -0.02017667,  0.00596662,
       -0.00049231, -0.009275  , -0.00573901, -0.00891541,  0.00074936,
       -0.04212594, -0.04092153,  0.03239125,  0.03318178, -0.0284719 ,
       -0.06044888, -0.0124798 ,  0.02722285, -0.05038751,  0.10497594,
        0.03214292,  0.0513551 , -0.02088493,  0.00735973, -0.02700034,
       -0.03710137,  0.00488788,  0.00363954, -0.00824471, -0.00886664,
       -0.0160233 , -0.07409945,  0.03489818,  0.06714744, -0.00168035,
       -0.06482008,  0.03922849,  0.03409362,  0.03135023,  0.07337287,
        0.00409382,  0.00085751,  0.06624596, -0.04060869,  0.01

In [ ]:
len(sale_and_weather_df[sale_and_weather_df['대분류_소분류'] == '식품_가공란'])

699

In [ ]:
#빈 list에다가 품목 카테고리별 날씨 correlation 추가
corr_table = []
corr_index = []
for i in category_array:
    #여기만 추가됨
    if len(sale_and_weather_df[sale_and_weather_df['대분류_소분류'] == i]) >= 365:
        corr_table.append(get_corr(i))
        corr_index.append(i)

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4174: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [ ]:
list = []

for i in category_array:
    

In [ ]:
#index = 품목 카테고리, colums = 날씨 변수로 해서 DataFrame으로 변환
corr_df = pd.DataFrame(corr_table, index = corr_index, columns = sale_and_weather_df.columns[2:])


In [ ]:
#판매량이 적어서 상관관계가 안나오는 경우도 존재(대략 220개 Na)해서 상관관계가 없다고(=0)으로 처리
corr_df.fillna(0, inplace = True)


In [ ]:
#index에 있는 품목 카테고리를 컬럼으로 빼고 reset index
corr_df = corr_df.rename_axis('대분류_소분류').reset_index()


In [ ]:
corr_df.head()

,대분류_소분류,평균기온(°C),최저기온(°C),최고기온(°C),1시간 최다강수량(mm),일강수량(mm),평균 풍속(m/s),풍정합(100m),최소 상대습도(%),평균 상대습도(%),O3,PM10,PM25,최다풍향북북동,최다풍향북동,최다풍향동북동,최다풍향동,최다풍향동남동,최다풍향남동,최다풍향남남동,최다풍향남,최다풍향남남서,최다풍향남서,최다풍향서남서,최다풍향서,최다풍향서북서,최다풍향북서,최다풍향북북서,최다풍향북,최고기온 시각(hh)0,최고기온 시각(hh)1,최고기온 시각(hh)2,최고기온 시각(hh)3,최고기온 시각(hh)4,최고기온 시각(hh)5,최고기온 시각(hh)6,최고기온 시각(hh)7,최고기온 시각(hh)8,최고기온 시각(hh)9,최고기온 시각(hh)10,...,1시간 최다 강수량 시각(hh)9.0,1시간 최다 강수량 시각(hh)10.0,1시간 최다 강수량 시각(hh)11.0,1시간 최다 강수량 시각(hh)12.0,1시간 최다 강수량 시각(hh)13.0,1시간 최다 강수량 시각(hh)14.0,1시간 최다 강수량 시각(hh)15.0,1시간 최다 강수량 시각(hh)16.0,1시간 최다 강수량 시각(hh)17.0,1시간 최다 강수량 시각(hh)18.0,1시간 최다 강수량 시각(hh)19.0,1시간 최다 강수량 시각(hh)20.0,1시간 최다 강수량 시각(hh)21.0,1시간 최다 강수량 시각(hh)22.0,1시간 최다 강수량 시각(hh)23.0,최소 상대습도 시각(hh)0,최소 상대습도 시각(hh)1,최소 상대습도 시각(hh)2,최소 상대습도 시각(hh)3,최소 상대습도 시각(hh)4,최소 상대습도 시각(hh)5,최소 상대습도 시각(hh)6,최소 상대습도 시각(hh)7,최소 상대습도 시각(hh)8,최소 상대습도 시각(hh)9,최소 상대습도 시각(hh)10,최소 상대습도 시각(hh)11,최소 상대습도 시각(hh)12,최소 상대습도 시각(hh)13,최소 상대습도 시각(hh)14,최소 상대습도 시각(hh)15,최소 상대습도 시각(hh)16,최소 상대습도 시각(hh)17,최소 상대습도 시각(hh)18,최소 상대습도 시각(hh)19,최소 상대습도 시각(hh)20,최소 상대습도 시각(hh)21,최소 상대습도 시각(hh)22,최소 상대습도 시각(hh)23,강수 계속시간(hr)
0,식품_가공란,-0.012727,-0.023704,-0.003491,0.033227,-0.001966,-0.032863,-0.033922,-0.072203,-0.045388,0.086238,0.149615,0.158268,-0.002307,-0.041985,-0.012814,-0.135865,-0.074622,0.003951,0.031752,0.001097,0.051458,0.050920,0.083184,0.072857,0.018187,-0.011568,0.030348,-0.077997,-0.020177,0.005967,-0.000492,-0.009275,-0.005739,-0.008915,0.000749,-0.042126,-0.040922,0.032391,0.033182,...,0.027758,0.030121,0.007324,0.040952,-0.065233,-0.023233,-0.069506,-0.009319,-0.024518,0.042314,0.019164,0.006746,-0.024718,0.019784,0.030358,-0.004213,0.001924,0.063422,-0.001633,0.010445,0.006145,-0.043432,-0.033885,-0.027981,0.032507,-0.028466,0.017325,-0.036507,0.009290,-0.038829,0.054500,-0.049070,0.093791,0.017824,-0.020740,-0.041671,0.029542,-0.012641,-0.022997,-0.039995
1,식품_옥수수,0.536916,0.554847,0.513040,0.217576,0.156799,-0.071048,-0.071511,0.424110,0.377176,0.211586,-0.261788,-0.154181,-0.178838,0.128091,0.048565,0.114959,0.042049,0.137806,0.054286,0.109572,0.234912,0.053081,0.128607,-0.068834,-0.111865,-0.164145,-0.171237,-0.100768,-0.014165,-0.037222,0.008021,0.029625,-0.026824,-0.023755,-0.020175,0.089996,0.055696,0.092985,0.048770,...,0.029049,0.051390,0.070368,0.009827,0.005786,0.007845,0.000689,0.148204,0.116172,0.061040,0.057996,0.070381,0.005374,0.030310,0.089783,-0.015092,-0.041925,0.038003,0.018330,0.065292,-0.048211,-0.024706,-0.015125,0.031423,0.007914,-0.072347,0.044677,-0.030367,-0.060164,0.026320,0.024791,0.023544,0.104428,0.043015,-0.012988,-0.081856,-0.024118,-0.027690,-0.040846,0.118374
2,식품_옥돔,-0.081266,-0.072173,-0.091446,0.015734,-0.024850,0.034962,0.036386,-0.003013,-0.037824,-0.080569,-0.012427,-0.040628,-0.020868,-0.040380,0.021475,-0.021120,-0.057610,-0.032932,0.055747,-0.070846,-0.065129,-0.030655,-0.047059,-0.019069,0.075710,0.038021,0.069159,0.067803,-0.002148,-0.008836,-0.021573,-0.033943,0.016429,-0.016791,-0.011504,-0.016587,-0.003763,-0.031547,0.003977,...,0.107030,0.070927,0.025190,0.072739,0.010438,0.021666,0.120452,-0.016067,-0.026593,0.083419,-0.027658,0.022932,0.045733,-0.038022,-0.023949,-0.039118,-0.019010,-0.004420,-0.035653,0.014046,0.066158,-0.021924,-0.013768,-0.056005,-0.031851,-0.012707,-0.024016,0.038037,-0.055043,-0.049678,0.116601,0.011346,0.027554,0.027139,0.017087,0.036426,-0.020500,-0.018177,0.004131,0.000883
3,식품_오징어,-0.104133,-0.102777,-0.102871,-0.035786,-0.047465,0.009401,0.009079,-0.094679,-0.105565,-0.049472,0.003306,0.015831,0.036498,-0.024129,-0.006636,-0.089502,-0.120590,-0.006292,-0.026143,-0.058762,0.048426,-0.040840,-0.003144,-0.045025,0.095329,0.094417,0.052754,-0.019613,-0.010887,-0.011065,-0.008311,-0.008316,-0.006595,-0.034180,0.016136,-0.001406,-0.038152,-0.025585,-0.040812,...,0.004408,-0.033503,-0.058016,0.033979,-0.030296,-0.033869,-0.018800,0.015508,0.015863,-0.017305,0.003890,-0.011156,-0.032903,-0.037129,0.032754,-0.024391,-0.015996,0.044409,-0.017397,0.027934,0.010377,

In [ ]:
#csv 파일로 저장
corr_df.to_csv('../data/sale_weather_correlation.csv', index = False, encoding='CP949')
